# 1. Intro
1. 这个父类`nn.module`模块就像一个树的head node一样，很多方法都要继承这个父类，比如loss或者conv等结构
2. 调用访问的两种方法:
    - 使用list: `list(net.parameters())`，想访问第0层就`list(net.parameters())[0]`
    - 使用dict: `dict(net.named_parameters()).items()`
# 2. children
1. 只有“直系亲属”才能叫做children，比如下面的`Sequential`，但是所有的relu这些，都是一个`module`
2. 根节点的含义还体现在`net.train()`，通过根节点进入train模式，它继承的所有node都会进入train模式

In [ ]:
from torch import nn
class BasicNet(nn.Module):
    def __init__(self):
        super(BasicNet,self).__init__()
        self.net = nn.Linear(4,3)
    def forward(self,x):
        return self.net(x)

class Net(nn.Module):
    def init_(self):
        super(Net,self).__init__=()
        # children
        self.net = nn.Sequential(BasicNet(),
                            nn.ReLU(),
                            nn.Linear(3,2))
    def forward(self,x):
        return self.net(x)

# 3. 加载自定义的class
1. 只有`class`才可以写进`Sequential`里面，`function`不行，如果某个操作，比如下面的`flatten`，只有`function`，没有`class`，那么可以**自己写**：

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten,self).__init__()
    def forward(self,input):
        return input.view(input.size(0),-1)

class TestNet(nn.Module):
    def __init__(self):
        super(TestNet,self).__init__()
        self.net = nn.Sequential(nn.Conv2d(1,16,kernel_size=1,stride=1,padding=1),
                                nn.MaxPool2d(2,2),
                                # 引入自定义flatten
                                Flatten(),
                                nn.Linear(1*14*14,10))
    def forward(self,x):
        return self.net(x)

2. 同理，可以设计更复杂的操作，下面就是自定义了linear层
    - `nn.Parameter`是一个包装器，对tensor进行包装，然后自动加入计算图，**并且加到`nn.parameters()`中去，从而可以被优化器进行自动优化**
    - `requires_grad = True`设置不设置都ok

In [ ]:
import torch
class MyLinear(nn.Module):
    def __init__(self,inp,outp):
        super(MyLinear,self).__init__()

        self.w=nn.Parameter(torch.randn(outp,inp))
        self.b=nn.Parameter(torch.randn(outp))
    def forward(self,x):
        x = x @ self.w.t() + self.b
        return x